In [9]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn
import torch.nn.functional as f
from torch.optim.lr_scheduler import ExponentialLR

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5), (0.5))])

batch_size = 4

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('0', '1', '2', '3',
           '4', '5', '6', '7', '8', '9')

/opt/anaconda3/lib/python3.7/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        break
inputs.shape

torch.Size([4, 1, 28, 28])

In [4]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 20, (3,3))
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(20, 25, (3, 3))
        self.conv3 = nn.Conv2d(25, 30, (3, 3))
        self.fc1 = nn.Linear(270, 10)

    def forward(self, x):
        x = self.pool(f.relu(self.conv1(x)))
        x = self.pool(f.relu(self.conv2(x)))
        x = f.relu(self.conv3(x))
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        return x

net = ConvNet()

In [5]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)
scheduler = ExponentialLR(optimizer, gamma=0.9)

In [6]:
for epoch in range(7):  

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
    scheduler.step()
    
print('Finished Training')

[1,  2000] loss: 0.360
[1,  4000] loss: 0.129
[1,  6000] loss: 0.102
[1,  8000] loss: 0.083
[1, 10000] loss: 0.077
[1, 12000] loss: 0.074
[1, 14000] loss: 0.067
[2,  2000] loss: 0.054
[2,  4000] loss: 0.052
[2,  6000] loss: 0.049
[2,  8000] loss: 0.056
[2, 10000] loss: 0.042
[2, 12000] loss: 0.044
[2, 14000] loss: 0.051
[3,  2000] loss: 0.029
[3,  4000] loss: 0.046
[3,  6000] loss: 0.036
[3,  8000] loss: 0.035
[3, 10000] loss: 0.031
[3, 12000] loss: 0.043
[3, 14000] loss: 0.034
[4,  2000] loss: 0.024
[4,  4000] loss: 0.023
[4,  6000] loss: 0.028
[4,  8000] loss: 0.035
[4, 10000] loss: 0.030
[4, 12000] loss: 0.028
[4, 14000] loss: 0.033
[5,  2000] loss: 0.015
[5,  4000] loss: 0.022
[5,  6000] loss: 0.020
[5,  8000] loss: 0.023
[5, 10000] loss: 0.021
[5, 12000] loss: 0.024
[5, 14000] loss: 0.023
[6,  2000] loss: 0.015
[6,  4000] loss: 0.015
[6,  6000] loss: 0.016
[6,  8000] loss: 0.018
[6, 10000] loss: 0.017
[6, 12000] loss: 0.016
[6, 14000] loss: 0.021
[7,  2000] loss: 0.013
[7,  4000] 

In [7]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images:', correct / total)

Accuracy of the network on the test images: 0.991


In [8]:
pytorch_total_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
pytorch_total_params

14215